In [43]:
import random
import math
import pandas as pd
import folium
import numpy as np
import geopandas 
import requests
import branca
# Example usage:
min_latitude = 30.0
max_latitude = 40.0
min_longitude = -120.0
max_longitude = -100.0
max_dist = 200
crowd_fraction = 0.8
crowd_centers = [(35.0, -115.0), (37.5, -110.0), (32.0, -105.0)]  # Example crowd centers
num_points = 500



def generate_random_points(min_lat, max_lat, min_lon, max_lon, crowd_centers, n):
    data_points = []

    for _ in range(n):

        if random.random() < crowd_fraction:
            # Select a random crowd center
            center = random.choice(crowd_centers)

            # Generate a random distance and angle
            distance = random.uniform(0, max_dist)  # Adjust the maximum distance as needed
            angle = random.uniform(0, 2 * math.pi)

            # Convert distance and angle to latitude and longitude differentials
            lat_diff = distance * math.cos(angle) / 111.32  # 1 degree of latitude is approximately 111.32 km
            lon_diff = distance * math.sin(angle) / (111.32 * math.cos(center[0]))

            # Calculate the new latitude and longitude
            new_lat = center[0] + lat_diff
            new_lon = center[1] + lon_diff

            # Ensure the generated point is within the specified boundaries
            new_lat = max(min_lat, min(new_lat, max_lat))
            new_lon = max(min_lon, min(new_lon, max_lon))

        else:
            new_lat = random.uniform(min_lat, max_lat)
            new_lon = random.uniform(min_lon, max_lon)

        data_points.append((new_lat, new_lon))

    return data_points


data_points = generate_random_points(min_latitude, max_latitude, min_longitude, max_longitude, crowd_centers, num_points)



In [44]:
data_points

[(37.4252091836137, -110.91584981926144),
 (35.84079441646386, -115.71222662432903),
 (31.705538658794254, -104.91544295032642),
 (39.72937476858231, -101.14006067697652),
 (36.41225573372696, -108.16340577493806),
 (36.78413674161234, -108.80803040341743),
 (37.670285536607814, -109.41652397574582),
 (31.899261639871437, -105.1995430769374),
 (33.6727275600505, -118.88240439507435),
 (35.57799387646673, -114.82974619130096),
 (38.16689925400308, -117.67622620790536),
 (31.5187932551396, -105.68096788754735),
 (35.08826841027063, -115.10366355864367),
 (37.881140119957934, -109.71685955948057),
 (32.040216193608515, -105.01531436504632),
 (37.525686431958135, -115.53365933367802),
 (36.55370631209793, -109.78449213326026),
 (35.202006100549475, -114.59961456449696),
 (37.174800124736606, -109.07082109878617),
 (37.29763893456065, -109.11836671996089),
 (32.18196782516668, -106.27052691923537),
 (37.748495685396364, -110.10518216231496),
 (34.730481329737614, -115.8744527204235),
 (34.5

In [45]:
import pandas as pd

# Your code for generating random points here...

# Create a DataFrame from the generated random points
data_points = pd.DataFrame(data_points, columns=['Latitude', 'Longitude'])

# Now, you have a DataFrame containing the generated data
data_points


,Latitude,Longitude
0,37.425209,-110.915850
1,35.840794,-115.712227
2,31.705539,-104.915443
3,39.729375,-101.140061
4,36.412256,-108.163406
...,...,...
495,36.144288,-114.699830
496,35.820977,-115.123475
497,32.116719,-103.126993
498,38.296807,-109.455414


In [46]:
truckdata = {
    'Longitude': [-96.8115, -100, -110, -95],
    'Latitude': [32.5202, 60, 50, 55]
}

truck_points = pd.DataFrame(truckdata)

# Print the DataFrame
truck_points


,Longitude,Latitude
0,-96.8115,32.5202
1,-100.0000,60.0000
2,-110.0000,50.0000
3,-95.0000,55.0000


In [47]:
data = requests.get(
    "https://raw.githubusercontent.com/python-visualization/folium-example-data/main/us_states.json"
).json()
states = geopandas.GeoDataFrame.from_features(data, crs="EPSG:4326")

states['strength'] = np.random.randint(0, 101, size=len(states))
states.head()

,geometry,name,strength
0,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",Alabama,53
1,"MULTIPOLYGON (((-131.60202 55.11798, -131.5691...",Alaska,40
2,"POLYGON ((-109.04250 37.00026, -109.04798 31.3...",Arizona,34
3,"POLYGON ((-94.47384 36.50186, -90.15254 36.496...",Arkansas,20
4,"POLYGON ((-123.23326 42.00619, -122.37885 42.0...",California,86


In [48]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
import random

# Define colormap for signal strength
colormap = branca.colormap.LinearColormap(
    vmin=states["strength"].quantile(0.0),
    vmax=states["strength"].quantile(1),
    colors=["red", "orange", "lightblue", "green", "darkgreen"],
    caption="Signal Strength",
)
# Create a Folium map centered on the USA
m = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

popup = folium.GeoJsonPopup(
    fields=["name", "strength"],
    aliases=["State", "% Change"],
    localize=True,
    labels=True,
    style="background-color: yellow;",
)

# Create a tooltip
tooltip = folium.GeoJsonTooltip(
    fields=["State", "strength"],
    aliases=["State:", "strength:"],
    localize=True,
    sticky=False,
    labels=True,
    style="font-weight: bold;",
    max_width=800,
)

# Create a GeoDataFrame with the 'State', 'Geometry', and 'Signal_Strength' columns
data = {'State': states['name'], 'geometry': states['geometry'], 'strength': states['strength']}
gdf = gpd.GeoDataFrame(data, geometry=states['geometry'], crs="EPSG:4326")


# Create a GeoJSON layer with style and tooltip
folium.GeoJson(
    gdf,
    style_function=lambda x: {
        "fillColor": colormap(x["properties"]["strength"])
        if x["properties"]["strength"] is not None else "transparent",
        "color": "black",
        "fillOpacity": 0.4,
    },
    tooltip=tooltip,
).add_to(m)

# Add colormap to the map
colormap.add_to(m)

# Save the map as an HTML file
m.save('usa_map.html')
m

In [49]:
# Create a map centered around the first location in your data
m = folium.Map(location=[data_points['Latitude'].iloc[0], data_points['Longitude'].iloc[0]], zoom_start=6,tiles="OpenStreetMap",control_scale =True)

# Add markers for each data point
data = folium.FeatureGroup().add_to(m)
for index, row in data_points.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']]).add_to(data)


truck = folium.FeatureGroup().add_to(m)
for index, row in truck_points.iterrows():
    folium.Marker(location = [row['Latitude'], row['Longitude']],icon = folium.plugins.BeautifyIcon(
    icon="truck", border_color="#b3334f", text_color="#b3334f", icon_shape="triangle"
)).add_to(truck)
    

folium.TileLayer("Stamen Watercolor").add_to(m)
folium.TileLayer("CartoDB Positron", show=False).add_to(m)

#folium.LatLngPopup().add_to(m)
m.fit_bounds(m.get_bounds())
folium.LayerControl().add_to(m)

# Display the map
m.save('map.html')  # Save the map to an HTML file
m


In [50]:
# Create a map centered around the first location in your data
m = folium.Map(location=[data_points['Latitude'].iloc[0], data_points['Longitude'].iloc[0]], zoom_start=6,tiles="OpenStreetMap",control_scale =True)

# Add markers for each data point
data = folium.FeatureGroup().add_to(m)
for index, row in data_points.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']]).add_to(data)

truck = folium.FeatureGroup().add_to(m)
for index, row in truck_points.iterrows():
    folium.Marker(location = [row['Latitude'], row['Longitude']],icon = folium.plugins.BeautifyIcon(
    icon="truck", border_color="#b3334f", text_color="#b3334f", icon_shape="triangle"
)).add_to(truck)
    
signal= folium.FeatureGroup().add_to(m)
# Add colormap to the map
# Create a GeoJSON layer with style and tooltip
folium.GeoJson(
    gdf,
    style_function=lambda x: {
        "fillColor": colormap(x["properties"]["strength"])
        if x["properties"]["strength"] is not None else "transparent",
        "color": "black",
        "fillOpacity": 0.4,
    },
    tooltip=tooltip,
).add_to(signal)
colormap.add_to(m)

folium.TileLayer("Stamen Watercolor").add_to(m)
folium.TileLayer("CartoDB Positron", show=False).add_to(m)

folium.LatLngPopup().add_to(m)
m.fit_bounds(m.get_bounds())
folium.LayerControl().add_to(m)

# Display the map
m.save('map.html')  # Save the map to an HTML file
m


In [51]:
from folium.plugins import HeatMap
# Create a map centered around the first location in your data
m = folium.Map(location=[data_points['Latitude'].iloc[0], data_points['Longitude'].iloc[0]], zoom_start=6)

# Create a HeatMap layer using population data
heat_data = [[row['Latitude'], row['Longitude']] for index, row in data_points.iterrows()]
HeatMap(heat_data).add_to(m)

# Display the map
m.save('population_heatmap.html')  # Save the map to an HTML file
m